In [1]:
import pandas as pd
import random

def create_subset_dataset(df, min_duration=0.3, max_duration=1.0):
    # Load the metadata file into a DataFrame
    # cols = ['filename', 'transcript', 'speaker', 'duration']
    # df = pd.read_csv(metadata_path, sep='|', names=cols, header=None)

    # Convert duration from seconds to hours
    df['duration_hours'] = df['duration'] / 3600

    # Group by speaker and calculate total duration
    grouped = df.groupby('speaker')['duration_hours'].sum().reset_index()
    
    # Filter speakers based on the duration range
    valid_speakers = grouped[(grouped['duration_hours'] >= min_duration) & (grouped['duration_hours'] <= max_duration)]['speaker']
    filtered_df = df[df['speaker'].isin(valid_speakers)]

    # Handle speakers with more than max_duration hours of audio
    excessive_speakers = grouped[grouped['duration_hours'] > max_duration]['speaker']

    for speaker in excessive_speakers:
        speaker_df = df[df['speaker'] == speaker]
        total_duration = speaker_df['duration_hours'].sum()
        
        while total_duration > max_duration:
            # Randomly remove a file
            to_remove = random.choice(speaker_df.index)
            total_duration -= speaker_df.loc[to_remove, 'duration_hours']
            speaker_df = speaker_df.drop(to_remove)
        
        filtered_df = pd.concat([filtered_df, speaker_df])

    # Save the final list of files to the output CSV file
    # filtered_df.drop(columns=['duration_hours'], inplace=True)
    # filtered_df.to_csv(output_path, sep='|', index=False, header=False)

    # print(f"Subset dataset saved to {output_path}")
    return filtered_df

# Example usage
metadata_path = "step14_tone_norm_transcript_no_multispeaker.txt"
output_path = "filtered_sach_noi_0.1_1h.txt"
# filtered_df = create_subset_dataset(metadata_path, output_path, min_duration=0.0, max_duration=1)

### load metadata

In [2]:
# long audio
metadata_path = "step14_tone_norm_transcript_no_multispeaker.txt"

# short audio to augment the dataset
# metadata_path = '/home/thivux/code/vinai/zstts/split_long_audio/metadata/step17_short_audio_wer0.csv'
cols = ['filename', 'transcript', 'speaker', 'duration', 'wer']
df = pd.read_csv(metadata_path, sep='|', names=cols, header=None)

In [3]:
df['duration'].sum() / 3600

1453.4473558427776

### create test data

top 20 speakers that speak the least will go to test set

In [4]:
test_speakers = [
    'Huỳnh_Minh_Hiền', 'Lê_Á_Thi', 'Hoàng_Tín', 'Chủ_Tịch_Hồ_Chí_Minh',
    'Nguyễn_Đình_Khánh', 'Thanh_Vân', '50_Nghệ_Sĩ-27-Huu Chau', 'BBC',
    'Thy_Lan', 'Nam_Anh', 'Nguyễn_Ngọc', '50_Nghệ_Sĩ-15-Ly Hung',
    'Thích_Chân_Tính', 'Hoàng_Mến', '50_Nghệ_Sĩ-39-Thai Hoa', '50_Nghệ_Sĩ',
    '50_Nghệ_Sĩ-10-Do Trung Quan', 'Hải_Khuê', '50_Nghệ_Sĩ-36-Tang Thanh Ha',
    'Lê_Bảo_Quốc'
]

test_df = df[df['speaker'].isin(test_speakers)]

train_df = df[~df['speaker'].isin(test_speakers)]

In [5]:
test_df['duration'].sum() / 60 # 0.4 hour
# drop the 'wer' column 
# test_df.drop(columns=['wer'], inplace=True)
# test_df.to_csv(metadata_path.replace('.csv', '_test.csv'), sep='|', index=False, header=False)

23.653989983333336

In [23]:
train_df.head()

,filename,transcript,speaker,duration,wer
0,big_processed_data/Nguyễn_Văn_Khỏa/Thần_Thoại_...,tôn kính thần linh. hay là họ là những con ngư...,Nguyễn_Văn_Khỏa,12.998563,NaN
1,big_processed_data/Nguyễn_Văn_Khỏa/Thần_Thoại_...,không phải là người biết ăn bánh mì. chúng sốn...,Nguyễn_Văn_Khỏa,11.391688,NaN
2,big_processed_data/Nguyễn_Văn_Khỏa/Thần_Thoại_...,ta sẽ chẳng xin phúc tha cho mày và đồng bọn c...,Nguyễn_Văn_Khỏa,10.878250,NaN
3,big_processed_data/Nguyễn_Văn_Khỏa/Thần_Thoại_...,từ ngoài khơi. khi con thuyền đã cần cập bến n...,Nguyễn_Văn_Khỏa,10.393875,NaN
4,big_processed_data/Nguyễn_Văn_Khỏa/Thần_Thoại_...,"giết chết hắn rồi, nhưng làm sao mà vần được c...",Nguyễn_Văn_Khỏa,10.795188,NaN


In [24]:
train_df.drop(columns=['wer'], inplace=True)
train_df.head()
# train_df.to_csv(metadata_path.replace('.csv', '_train.csv'), sep='|', index=False, header=False)
train_df['duration'].sum() / 3600 # 1453 hour

/tmp/ipykernel_56430/4059279811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(columns=['wer'], inplace=True)


1453.0531226763887

In [25]:
filtered_train = create_subset_dataset(train_df, min_duration=0.0, max_duration=4)

/tmp/ipykernel_56430/22099980.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['duration_hours'] = df['duration'] / 3600


In [30]:
filtered_train['duration'].sum() / 3600 

939.9035293661113

In [8]:
train_speakers = filtered_train.groupby('speaker').sum('duration') / 3600
train_speakers

,duration,duration_hours
speaker,,
50_Nghệ_Sĩ-01-Quyen Linh,0.069887,0.000019
50_Nghệ_Sĩ-02-Ngo Thanh Van,0.045821,0.000013
50_Nghệ_Sĩ-03-Thien Ly,0.036944,0.000010
50_Nghệ_Sĩ-04-Dam Vinh Hung,0.035599,0.000010
50_Nghệ_Sĩ-05-Trung Dung,0.031720,0.000009
...,...,...
Đỗ_Thụy,1.124952,0.000312
Độc_Thám_TV,2.449838,0.000681
Đức_Huy,0.566308,0.000157


In [10]:
filtered_train.head()

,filename,transcript,speaker,duration,wer,duration_hours
0,big_processed_data/Lê_Đức_Quân/Đối_Thoại_Với_M...,nó sẽ đi thôi.,Lê_Đức_Quân,1.36,0.0,0.000378
1,big_processed_data/Nguyễn_Văn_Khỏa/Thần_Thoại_...,"chúng đem bán, mỗi người mỗi nơi",Nguyễn_Văn_Khỏa,2.90,0.0,0.000806
2,big_processed_data/Meredith_Mclver/Trump_101:_...,"tuy nhiên, ở nơi làm việc, sự an toàn sẽ kìm h...",Meredith_Mclver,3.77,0.0,0.001047
3,big_processed_data/Phúc_Lâm/Sức_Mạnh_Của_Động_...,mà chính họ gặp phải thông qua phương pháp này.,Phúc_Lâm,2.94,0.0,0.000817
4,big_processed_data/Diễm_Hân/Nhân_Tố_Enzyme/nha...,nhưng khi cho thêm vào nước thì một lượng lớn ...,Diễm_Hân,5.48,0.0,0.001522


In [11]:
test_df.head()

,filename,transcript,speaker,duration,wer
753292,big_processed_data/50_Nghệ_Sĩ/50_Nghệ_Sĩ_Đọc_“...,cho nên,50_Nghệ_Sĩ,1.26,0.0
753295,big_processed_data/50_Nghệ_Sĩ/50_Nghệ_Sĩ_Đọc_“...,"bi quan, chán nản,",50_Nghệ_Sĩ,1.20,0.0
753296,big_processed_data/50_Nghệ_Sĩ/50_Nghệ_Sĩ_Đọc_“...,thì họ sẽ được cứu rỗi về một thế giới chỉ toà...,50_Nghệ_Sĩ,3.90,0.0
753297,big_processed_data/50_Nghệ_Sĩ/50_Nghệ_Sĩ_Đọc_“...,cũng là một kiểu mê tín.,50_Nghệ_Sĩ,2.06,0.0
753298,big_processed_data/50_Nghệ_Sĩ/50_Nghệ_Sĩ_Đọc_“...,đó là bởi ta không biết,50_Nghệ_Sĩ,2.52,0.0


In [9]:
# concat filtered_train & test_df and save to txt file 
filtered_df = pd.concat([filtered_train, test_df], ignore_index=True)
df_to_save = filtered_df.drop(columns=['duration_hours'])
# df_to_save.to_csv("step16_cap_speaker_length_940h.txt", sep='|', index=False, header=False)

In [10]:
filtered_df['duration'].sum() / 3600

554.1057833333332

In [24]:
# save train & test to separate files
filtered_train.to_csv("step16_cap_speaker_length_940h_train.txt", sep='|', index=False, header=False)
test_df.to_csv("step16_cap_speaker_length_940h_test.txt", sep='|', index=False, header=False)

### create test set of 20 seen speakers in sachnoi

In [34]:
# df of samples in train_df but not in filtered_train
diff_df = train_df.merge(filtered_train, on=['filename', 'transcript', 'speaker', 'duration', 'duration_hours'], 
                         how='left', indicator=True)

# Filter rows that are only in train_df (i.e., not in filtered_train)
left_out = diff_df[diff_df['_merge'] == 'left_only']

# Drop the '_merge' column
left_out = left_out.drop(columns=['_merge'])

# Display the result
left_out.head()

,filename,transcript,speaker,duration,duration_hours
302,big_processed_data/Ngọc_Như/Thảm_Kịch_Bí_Ẩn_Ở_...,"phải không? mười năm rồi, thưa ông. đó là một ...",Ngọc_Như,11.114063,0.003087
313,big_processed_data/Ngọc_Như/Thảm_Kịch_Bí_Ẩn_Ở_...,ông ta cầm chiếc nón kết trên tay và lặng lẽ x...,Ngọc_Như,11.217188,0.003116
322,big_processed_data/Ngọc_Như/Thảm_Kịch_Bí_Ẩn_Ở_...,ông có vẻ ăn năn đến độ tôi thấy tội nghiệp ch...,Ngọc_Như,10.590625,0.002942
328,big_processed_data/Ngọc_Như/Thảm_Kịch_Bí_Ẩn_Ở_...,bà ta đang thay quần áo thì bị kinh hãi bởi ti...,Ngọc_Như,11.173438,0.003104
329,big_processed_data/Ngọc_Như/Thảm_Kịch_Bí_Ẩn_Ở_...,"vâng, đúng như vậy và cửa sổ thư phòng thì lại...",Ngọc_Như,10.200250,0.002833


In [35]:
# drop the duration_hours column
left_out.drop(columns=['duration_hours'], inplace=True)
left_out.head()


,filename,transcript,speaker,duration
302,big_processed_data/Ngọc_Như/Thảm_Kịch_Bí_Ẩn_Ở_...,"phải không? mười năm rồi, thưa ông. đó là một ...",Ngọc_Như,11.114063
313,big_processed_data/Ngọc_Như/Thảm_Kịch_Bí_Ẩn_Ở_...,ông ta cầm chiếc nón kết trên tay và lặng lẽ x...,Ngọc_Như,11.217188
322,big_processed_data/Ngọc_Như/Thảm_Kịch_Bí_Ẩn_Ở_...,ông có vẻ ăn năn đến độ tôi thấy tội nghiệp ch...,Ngọc_Như,10.590625
328,big_processed_data/Ngọc_Như/Thảm_Kịch_Bí_Ẩn_Ở_...,bà ta đang thay quần áo thì bị kinh hãi bởi ti...,Ngọc_Như,11.173438
329,big_processed_data/Ngọc_Như/Thảm_Kịch_Bí_Ẩn_Ở_...,"vâng, đúng như vậy và cửa sổ thư phòng thì lại...",Ngọc_Như,10.200250


In [38]:
import numpy as np

# Step 1: Randomly sample 20 unique speakers from left_out
sampled_speakers = np.random.choice(left_out['speaker'].unique(), size=20, replace=False)

# Step 2: Filter left_out for only the sampled speakers
left_out_sampled = left_out[left_out['speaker'].isin(sampled_speakers)]

# Step 3: Create an empty list to store the filtered rows
filtered_rows = []

# Group by speaker and accumulate duration up to 3 minutes (180 seconds)
grouped = left_out_sampled.groupby('speaker')

for speaker, group in grouped:
    # Sort by duration in descending order to take the longer samples first
    
    cumulative_duration = 0
    for idx, row in group.iterrows():
        if cumulative_duration + row['duration'] > 180:  # 180 seconds = 3 minutes
            break
        filtered_rows.append(row)
        cumulative_duration += row['duration']

# Convert the filtered rows back to a DataFrame
left_out_subset = pd.DataFrame(filtered_rows)

# Display the result
left_out_subset.head()

,filename,transcript,speaker,duration
137961,big_processed_data/Dương_Liễu/Truyện_Ngụ_Ngôn_...,"lòng ghen tị của ếch nổi lên, nó ước sao trở t...",Dương_Liễu,13.720000
137962,big_processed_data/Dương_Liễu/Truyện_Ngụ_Ngôn_...,"nếu các cháu còn cứ ở đó, thì những bụi cây ga...",Dương_Liễu,11.196875
137966,big_processed_data/Dương_Liễu/Truyện_Ngụ_Ngôn_...,thế thì đã ăn nhầm gì? chó nhà đáp. thế còn gì...,Dương_Liễu,14.577188
137967,big_processed_data/Dương_Liễu/Truyện_Ngụ_Ngôn_...,bởi vì nơi tôi uống nước còn cách xa chỗ ngài ...,Dương_Liễu,12.679063
137968,big_processed_data/Dương_Liễu/Truyện_Ngụ_Ngôn_...,"thế là, con sói cắp chứa cừu non vào tận rừng ...",Dương_Liễu,13.003625


In [39]:
left_out_subset['duration'].sum() / 3600

0.962697553888889

In [40]:
# get speakers in left_out_subset
left_out_subset_speakers = left_out_subset['speaker'].unique()
# make sure they appear in train set (filtered_train)
for speaker in left_out_subset_speakers:
    assert speaker in filtered_train['speaker'].unique()

In [ ]:
# save left_out_subset to csv file
left_out_subset.to_csv("sachnoi_seen_speaker_test.csv", sep='|', index=False, header=False)

In [44]:
# create json files for test set (left_out_subset)
# each speaker = 1 json file. each json file = many samples, each sample = {path, duration, transcript}
import os
import json

# Define the output folder for JSON files
json_folder = 'json_files_sachnoi_seen_test'
os.makedirs(json_folder, exist_ok=True)  # Create the folder if it doesn't exist

# Group by speaker and create a JSON file for each speaker
grouped = left_out_subset.groupby('speaker')

lustre_path = "/lustre/scratch/client/vinai/users/thivt1/code/oneshot"
for speaker, group in grouped:
    # Create a list of samples for the speaker
    samples = []
    for _, row in group.iterrows():
        sample = {
            'path': os.path.join(lustre_path, row['filename']),  # Adjust the key to match your DataFrame
            'duration': row['duration'],
            'transcript': row['transcript']
        }
        samples.append(sample)
    
    # Define the JSON file name
    json_file_path = os.path.join(json_folder, f"sachnoi-seen-{speaker}.json")
    
    # Write samples to the JSON file
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(samples, json_file, ensure_ascii=False, indent=4)

print(f"JSON files created in {json_folder}")


JSON files created in json_files_sachnoi_seen_test


### save to json file

In [ ]:
import os
import json


train_path = "sach_noi_train.json"
test_path = 'sach_noi_test.json'
root_dir = "/lustre/scratch/client/vinai/users/thivt1/code/oneshot"

# Load dialect information from JSONL file
dialect_file = "/lustre/scratch/client/vinai/users/thivt1/code/oneshot/data_stories_large_model.jsonl"
dialect_info = {}

with open(dialect_file, 'r') as f:
    for line in f:
        data = json.loads(line)
        dialect_info[data['path']] = data['dialect']['Label']

# Function to create dictionary from DataFrame row
def create_dict(row):
    path = os.path.join(root_dir, row['filename'])
    dialect = dialect_info.get(path)
    if dialect:
        return {
            "path": path,
            "transcript": row['transcript'],
            "speaker": row['speaker'],
            "duration": row['duration'],
            "dialect": dialect,
            "segment_id": get_segment_id_from_path(row['filename'])
        }
    else: 
        raise ValueError(f"Dialect information not found for {path}")

def get_segment_id_from_path(path):
    if len(path.split("/")[1:]) == 3:
        return "___".join(path.replace('.wav', '').split("/")[1:])
    else: 
        speaker, book, chapter, segment = path.replace('.wav', '').split("/")[1:]
        return f"{speaker}___{chapter}___{segment}"

# Create list of dictionaries for train and test sets
train_list = [create_dict(row) for index, row in train_df.iterrows()]
test_list = [create_dict(row) for index, row in test_df.iterrows()]

# Save to JSON files
with open(train_path, 'w', encoding='utf-8') as f:
    json.dump(train_list, f, ensure_ascii=False, indent=4)

with open(test_path, 'w', encoding='utf-8') as f:
    json.dump(test_list, f, ensure_ascii=False, indent=4)

print("JSON files created successfully.")

In [48]:
import json

with open('json_files_sachnoi_seen_test/sachnoi-seen-Dương_Liễu.json', 'r') as f:
    data = json.load(f)
    
data[0]

{'path': '/lustre/scratch/client/vinai/users/thivt1/code/oneshot/big_processed_data/Dương_Liễu/Truyện_Ngụ_Ngôn_La_Fontaine/truyen-ngu-1/chunk-268_82-282_44_trimmed_norm_float32.wav',
 'duration': 13.72,
 'transcript': 'lòng ghen tị của ếch nổi lên, nó ước sao trở thành to lớn như bò kia. nó cố gắng phình người ra, bành cổ rộng ra, hy vọng cải thiện được vóc dáng bé nhỏ của mình.'}